# Setup
For this experiment, an AWS account is required. The services used are namely AWS Simple Storage Service (S3), AWS Elastic Compute (EC2), and AWS CloudFront CDN. Running this experiment may incur slight charges on your AWS account.

### Request Target


We'll create a 256KB file for our experiment. This same file will be the request target for all three content delivery methods.
```{bash}
$ dd if=/dev/zero of=payload  bs=256KB  count=1
```

## S3

First, upload the payload file to an S3 bucket. Detailed instructions on how to upload files to S3 can be found here https://docs.aws.amazon.com/AmazonS3/latest/user-guide/upload-objects.html

Take note of the following when uploading the object to S3:
* The bucket should be hosted in the *ap-southeast-1* region which is located in Singapore
* Public access to the object is disallowed by default. Turn on public access to the file.
* Choose **S3 Standard** for the object storage class

Once done, the file can be accessed over the Internet via the URL format:
```{bash}
https://<bucket-name>.s3-ap-southeast-1.amazonaws.com/payload
```

## CloudFront CDN

Next, with the object uploaded to S3, you can easily enable the CDN service on the S3 bucket. This will allow the same file to be served through Amazon's content delivery network - CloudFront. The CDN url path for the payload object should be in the format below:
```{bash}
https://<your-assigned-cloudfront-url>.cloudfront.net/payload
```
Detailed instructions on how to enable CloudFront on S3 can be found here, and it should take only a few minutes setup: https://aws.amazon.com/premiumsupport/knowledge-center/cloudfront-https-requests-s3/

## NGINX File Server

We will first deploy an AWS EC2 instance and use NGINX as a static file server to serve the payload file. If you have never deployed an EC2 instance before, you can follow the instructions here:
https://docs.aws.amazon.com/efs/latest/ug/gs-step-one-create-ec2-resources.html

 We will deploy a *t2.small* instance using the standard *Amazon Linux 2 AMI (HVM), SSD Volume Type* AMI. <br>
 Take note that of the following when launching this instance:
 * Instance should be deployed in the **ap-southeast-1a** availability zone in the Singapore region.  
 * For simplicity, you may deploy this instance in the default subnet.
 * Configure the security group to allow inbound traffic from port 80
 * Input user-data script below

#### Bootstrap Script

When deploying EC2 instances, we can upload bash scripts during deployment and these scripts will run when the instance is deployed automatically. To automate the server configurations, we will add the configuration script to run on instance launch. To find out more on running scripts on instance launch, see the docs here: https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/user-data.html

The following script goes into the **user-data** field in *Step 3 - Configure Instance* in the AWS Console when launching an EC2 instance:

```{bash}
#!/bin/bash
yum update -y
amazon-linux-extras install nginx1.12 -y
cat > local.conf <<- EOM
server {
    listen 80;
    server_name _;
    root /usr/share/nginx;
    location / {
    }
}
EOM
mv local.conf /etc/nginx/conf.d/local.conf
dd if=/dev/zero of=/usr/share/nginx/html/payload  bs=256KB  count=1
systemctl restart nginx

```

What this script does is:
1. Install Nginx
2. Configure Nginx to serve files on port 80
3. Create the 256KB payload file 
4. Restarts Nginx

The file server is now ready and the payload file will be accessible in the given URL format:
```{bash}
http://<public-ipv4-address>/payload
```

Overwrite the URLS to the 3 file servers in *config.py* 

In [4]:
# Replace key values in config.py
source = {
    "CLOUDFRONT": "https://<your-assigned-cloudfront-url>.cloudfront.net/payload",
    "S3": "https://<bucket-name>.s3-ap-southeast-1.amazonaws.com/payload",
    "NGINX": "http://<public-ipv4-address>/payload",
}

Now all three content delivery methods are ready and we can begin the experiment.<br>